<a href="https://colab.research.google.com/github/rafapaez/Leadership-Mentor-Agent/blob/main/Leadership_Mentor_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Libraries Installation ---
# First, ensure you have the necessary library installed:
!pip install litellm --quiet

import os
import json
from litellm import completion
from google.colab import userdata

# --- Agent Configuration ---
# Securely manage your API key by setting a environment variable:
api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

class LeadershipMentorAgent:
    """
    A simple AI Agent that acts as a leadership mentor.
    """
    def __init__(self, goal):
        """
        Initializes the agent with a goal, memory, and a set of tools.
        """
        self.goal = goal
        self.memory = []
        self.max_loops = 10 # Set a maximum number of interactions
        # A registry of available "mentoring" tools for the agent.
        self.tools = {
            "clarify": {
                "function": self.clarify,
                "description": "Asks the user a question to get more details about their goal. Use this if the goal is vague.",
                "args": {"question": "The clarifying question to ask the user (string)."}
            },
            "teach": {
                "function": self.teach,
                "description": "Explains a leadership concept, theory, or strategy to the user. Use this to provide knowledge.",
                "args": {"concept": "The concept to explain (string)."}
            },
            "evaluate": {
                "function": self.evaluate,
                "description": "Asks the user a question to see if they understood a concept. Use this to check for comprehension.",
                "args": {"question": "The question to evaluate understanding (string)."}
            },
            "finish": {
                "function": self.finish,
                "description": "Ends the mentoring session with a summary or concluding remarks.",
                "args": {"summary": "A final message for the user (string)."}
            }
        }

    # These functions define the agent's "actions". They interact with the user via the console.
    def clarify(self, question: str) -> str:
        """Asks the user for more information."""
        print(f"🤖 Mentor: {question}")
        user_response = input("You: ")
        return f"User's response: '{user_response}'"

    def teach(self, concept: str) -> str:
        """Explains a concept to the user."""
        # In a real app, you might fetch this from a knowledge base.
        print(f"🤖 Mentor: Let's talk about '{concept}'. A key aspect of this is...")
        # Mock explanation
        if "feedback" in concept.lower():
            print("Effective feedback is specific, actionable, and delivered with empathy. It focuses on behavior, not personality.")
        else:
            print("It's important for a leader to understand this principle to guide their team effectively.")
        return "Concept explained."

    def evaluate(self, question: str) -> str:
        """Checks if the user understood the concept."""
        print(f"🤖 Mentor: {question}")
        user_response = input("You: ")
        return f"User's response to evaluation: '{user_response}'"

    def finish(self, summary: str) -> str:
        """Ends the conversation."""
        print(f"🤖 Mentor: {summary}")
        return "Session concluded."

    def _construct_prompt(self) -> list[dict]:
        """
        Constructs the prompt for the LLM with instructions, tools, and history.
        """
        # The system prompt defines the agent's persona and logic.
        system_prompt = f"""
        You are an AI Leadership Mentor. Your purpose is to help the user with their goal: "{self.goal}".
        You will guide them by asking questions, teaching concepts, and evaluating their understanding.

        You have access to the following tools:
        {json.dumps(self.tools, indent=2, default=lambda o: o.__doc__)}

        Follow this thought process:
        1.  Start by understanding the user's goal. If it's unclear, use 'clarify'.
        2.  Based on their goal, use 'teach' to explain a relevant leadership skill.
        3.  After teaching, use 'evaluate' to check if they grasped the concept.
        4.  Based on their responses, decide the next step. Either teach a new concept or continue clarifying.
        5.  When the user's initial goal has been addressed, use 'finish' to end the conversation.
        6.  Your responses MUST be in a valid JSON format.

        Example of a tool-using response:
        {{
            "action": "teach",
            "args": {{"concept": "Giving constructive feedback"}}
        }}

        Example of a concluding response:
        {{
            "action": "finish",
            "args": {{"summary": "Great work today! We've covered the basics of feedback."}}
        }}
        """

        full_prompt = [{"role": "system", "content": system_prompt}]
        full_prompt.extend(self.memory)
        # Add the goal as the first user message if memory is empty
        if not self.memory:
            full_prompt.append({"role": "user", "content": f"My leadership goal is: {self.goal}"})

        return full_prompt

    def run(self):
        """
        The main agent loop for the mentoring session.
        """
        print(f"🎯 Mentor Agent activated. Goal: {self.goal}\n")
        loop_count = 0
        while loop_count < self.max_loops:
            loop_count += 1
            print(f"--- Interaction {loop_count}/{self.max_loops} ---")

            # 1. Construct the prompt
            prompt = self._construct_prompt()

            # 2. Query the LLM for the next action
            print("🤔 Mentor is thinking...")
            try:
                response = completion(
                    model="gpt-4o", # Using a more capable model for nuanced tasks
                    messages=prompt,
                    temperature=0.7,
                    max_tokens=512
                )
                response_content = response.choices[0].message.content.strip()
            except Exception as e:
                print(f"An error occurred with the LLM call: {e}")
                break

            self.memory.append({"role": "assistant", "content": response_content})

            # 3. Parse the LLM's response
            try:
                parsed_response = json.loads(response_content)
                print(f"✅ Mentor's decided action: {parsed_response['action']}")
            except (json.JSONDecodeError, KeyError):
                print(f"🚨 Error: Invalid JSON from LLM: {response_content}")
                feedback = "Invalid response format. Please use the 'action' key in a JSON object."
                self.memory.append({"role": "system", "content": f"Feedback: {feedback}"})
                continue

            # 4. Execute the chosen action
            action_name = parsed_response.get("action")
            action_args = parsed_response.get("args", {})

            if action_name in self.tools:
                tool_function = self.tools[action_name]["function"]
                try:
                    result = tool_function(**action_args)
                    # 5. Capture feedback for the next loop
                    feedback = f"The user's response was: '{result}'"
                    print(f"🔧 Feedback captured.\n")
                    self.memory.append({"role": "system", "content": f"Feedback: {feedback}"})

                    # 6. Decide to continue or end
                    if action_name == "finish":
                        print("\n🏁 Mentor session has ended.")
                        break
                except Exception as e:
                    print(f"Error executing tool '{action_name}': {e}")
                    break
            else:
                feedback = f"Unknown tool '{action_name}'. Please choose from available tools."
                self.memory.append({"role": "system", "content": f"Feedback: {feedback}"})

        if loop_count >= self.max_loops:
            print("\n⚠️ Mentor reached maximum interactions. Ending session.")

# --- Main Execution ---
if __name__ == "__main__":
    try:
        user_goal = input("Welcome! What leadership goal would you like to work on today?\nYou: ")
        if user_goal:
            agent = LeadershipMentorAgent(goal=user_goal)
            agent.run()
        else:
            print("No goal provided. Exiting.")
    except KeyboardInterrupt:
        print("\nProcess interrupted by user. Exiting.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 71.2 MB/s eta 0:00:00
Welcome! What leadership goal would you like to work on today?
You: influence
🎯 Mentor Agent activated. Goal: influence

--- Interaction 1/10 ---
🤔 Mentor is thinking...
✅ Mentor's decided action: clarify
🤖 Mentor: Can you tell me more about the specific context or area where you want to increase your influence? Is it within your team, organization, or externally?
You: at work
🔧 Feedback captured.

--- Interaction 2/10 ---
🤔 Mentor is thinking...
✅ Mentor's decided action: teach
🤖 Mentor: Let's talk about 'One effective way to increase your influence at work is by building strong relationships. This includes understanding your colleagues' perspectives, actively listening, and showing empathy. By doing so, you can create a network of support and trust, which is crucial for influence.'. A key aspect of this is...
It's important for a leader to understand this principle to guide their team effectively.
🔧 Feedback